<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/our%20framework%20code/answer_Generation_Llama3_1_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# stting up

In [ ]:
!pip install datasets
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens 

In [ ]:
import os
import torch
import json
import pandas as pd
from google.colab import files
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from IPython.display import FileLink, display
from datasets import load_dataset

#**loading the model**

In [ ]:
# Load the model and tokenizer
model_id = "meta-llama/Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Create the text generation pipeline
pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer
)

# Test the pipeline
print(pipeline("من اسس السعودية الثالثة؟"))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'من اسس السعودية الثالثة؟ 2014-09-22 10:41:05\nمن هو الخليفة الر'}]


# helper functions

In [ ]:
torch.cuda.empty_cache()

with the likelihood:)

In [ ]:
def generate_answer(question, num_samples, model, tokenizer):
    # Create the prompt with the question
    prompt = f" أجب على السؤال التالي بجملة واحدة فقط موجزة ولكن كاملة باللغة العربية\nQuestion: {question}\nAnswer:"

    # Tokenize the prompt without adding special tokens
    prompt_tokens = tokenizer(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)
    prompt_length = prompt_tokens.input_ids.shape[1]  # Get actual prompt length

    # Generate responses using the model
    outputs = model.generate(
        **prompt_tokens,
        max_length=150,
        do_sample=True,
        temperature=0.5,
        top_p=0.95,
        num_return_sequences=num_samples,
        return_dict_in_generate=True,
        output_scores=True,
    )

    sequences = outputs.sequences
    results = []

    for seq in sequences:
        # Decode and clean the generated text
        decoded_text = tokenizer.decode(seq, skip_special_tokens=True)

        # Extract the answer after the "Answer:" marker
        answer = decoded_text.split("Answer:")[-1].strip()

        #clean the output like what they did
        answer = decoded_text.split("Answer:")[-1].strip()
        strings_to_filter_on = ['.', '\n', 'Q:', 'A:', 'question:', 'answer:', 'Question:', 'Answer:', 'Questions:', 'questions:', 'QUESTION:', 'ANSWER:']
        for string in strings_to_filter_on:
            if string in answer:
                answer = answer.split(string)[0].strip()

        # Calculate negative log-likelihood
        with torch.no_grad():
            # Create target labels with prompt masked
            target_ids = seq.clone()
            target_ids[:prompt_length] = -100  # Ignore prompt tokens in loss

            # Forward pass to compute loss
            output = model(seq.unsqueeze(0), labels=target_ids.unsqueeze(0))

            # Calculate total NLL for generated tokens
            avg_nll_per_token = output.loss
            num_generated_tokens = len(seq) - prompt_length
            total_nll = avg_nll_per_token * num_generated_tokens

        results.append({
            'text': answer,
            'neg_log_likelihood': total_nll.item(),
        })

    return results

In [ ]:
def load_qa_dataset(dataset_name, file_path=None):
    data = []

    try:
        if dataset_name == 'arabicaqa' and file_path:
            if os.path.exists(file_path):  # Use the loaded file
                print("Using ArabicaQA")
                with open(file_path, 'r', encoding='utf-8') as f:
                    custom_data = json.load(f)
                    for idx, item in enumerate(custom_data):
                        data.append({
                            "question_id": idx,
                            "Question": item["question"],
                            "Answer": item["answer"]
                        })
            else:
                raise FileNotFoundError(f"ArabicaQA file not found at {file_path}")

        elif dataset_name == 'xor_tydiqa' and file_path:
            if os.path.exists(file_path):  # Use the loaded file
                print("Using XOR-TyDiQA")
                print("Filtering Arabic QA pairs from XOR-TyDi...")

                # Load the jsonl dataset
                with open(file_path, 'r', encoding='utf-8') as f:
                    custom_data = [json.loads(line) for line in f]  # Handling jsonl

                # Filter Arabic samples ("lang" = "ar")
                arabic_data = []
                for idx, item in enumerate(custom_data):
                    if item["lang"] == "ar":  # Arabic language code
                        arabic_data.append({
                            "question_id": item["id"],
                            "Question": item["question"],
                            "Answer": item["answers"][0]  # First answer in list
                        })

                data.extend(arabic_data)
            else:
                raise FileNotFoundError(f"XOR-TyDiQA file not found at {file_path}")

        elif dataset_name == 'xquad':
            print("Using XQuAD")
            dataset = load_dataset("xquad", "xquad.ar")
            for idx, item in enumerate(dataset["validation"]):  # Only available set
                data.append({
                    "question_id": f"xquad_{idx}",
                    "Question": item["question"],
                    "Answer": item["answers"]["text"][0]
                })

        elif dataset_name == 'arcd':
            print("Using ARCD")
            dataset = load_dataset("arcd")
            for split in ["train", "validation"]:
                for idx, item in enumerate(dataset[split]):
                    data.append({
                        "question_id": f"arcd_{idx}",
                        "Question": item["question"],
                        "Answer": item["answers"]
                    })

        elif dataset_name == 'arabic_squad':
            print("Using Arabic SQuAD")
            dataset = load_dataset("Mostafa3zazi/Arabic_SQuAD")
            for split in ["train"]:
                for idx, item in enumerate(dataset[split]):
                    data.append({
                        "question_id": f"squad_{idx}",
                        "Question": item["question"],
                        "Answer": item["text"]
                    })

        elif dataset_name == 'mintaka':
            print(f"Using Mintaka in arabic")
            dataset = load_dataset("AmazonScience/mintaka", "ar")
            if "train" in dataset:
                dataset = dataset["train"]  # Select training split
            for idx, item in enumerate(dataset):
                    data.append({
                        "question_id": item['id'],
                        "Question": item['question'],
                        "Answer": item['answerText']
                    })

        elif dataset_name == 'aqad' and file_path:
            if os.path.exists(file_path):
                print("Using AQAD")
                # Load the dataset
                with open(file_path, 'r', encoding='utf-8') as f:
                    dataset = json.load(f)  # Assuming the dataset is in JSON format

                # Loop through the data structure
                for article in dataset.get('data', []):  # Ensure to check 'data'
                    title = article.get('title', 'Unknown Title')  # Handle missing title
                    for paragraph in article.get('paragraphs', []):  # Ensure to check 'paragraphs'
                        for qas in paragraph.get('qas', []):  # Ensure to check 'qas'
                            question = qas.get('question', '').strip()  # Handle missing question
                            answers = qas.get('answers', [])
                            if answers:
                                answer_text = answers[0].get('text', '').strip()  # Handle missing answer
                                if question and answer_text:  # Ensure both question and answer are present
                                    data.append({
                                        "question_id": f"aqad_{qas.get('id', 'unknown')}",  # Handle missing ID
                                        "Question": question,
                                        "Answer": answer_text
                                    })
                            else:
                                print(f"Skipping entry with missing answer for question: {question}")

        else:
            raise ValueError(f"Unsupported dataset: {dataset_name}")

    except Exception as e:
        print(f"Error loading {dataset_name}: {str(e)}")
        return []

    return data

# main function to to load questions and gerate answers and save them to a file

In [ ]:
 # Main function
def main():
    # Step 1: SELECT AND LOAD DATA
    DATASET_CHOICE = 'xor_tydiqa'  # Choose from above

    FILE_PATH = '/content/xor_dev_full_v1_1.jsonl'  # Only needed for ArabicaQA And xor_tydiqa and AAQAD

    data = load_qa_dataset(DATASET_CHOICE, FILE_PATH if DATASET_CHOICE == 'arabicaqa' or 'xor_tydiqa' else None)

    if not data:
        print("No data loaded. Exiting...")
        return

    # Convert to DataFrame
    df = pd.DataFrame(data)[["question_id", "Question", "Answer"]]
    print(f"Loaded {len(df)} questions\n")

  # Step 2: GENERATE ANSWERS
  #empty list to store the results
    results = []

    for index, row in df.iterrows():
        print(f"Processing question {index+1}/{len(df)}")
        question = row['Question']
        original_answer = row.get('Answer',"")  # Use empty string if no original answer provided
        question_id = row.get('question_id')

        # Generate 10 answers with log probabilities for each question
        generated_results = generate_answer(
            question=question,
            num_samples=10,
            model=model,
            tokenizer=tokenizer
        )
        # Extract text answers and log probabilities
        # Append the results

        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": row.get('Answer', ""),
            "generated_answers": [
                {"text": r['text'], "neg_log_likelihood": r['neg_log_likelihood']}
                for r in generated_results
            ]
        })

   # Step 3: SAVE RESULTS
    results_df = pd.DataFrame(results)
    print(results_df)

    filename = f"generated_answers_Llama3.1-8b{DATASET_CHOICE}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    print(f"\nSaved results to {filename}")
    files.download(filename)

# Run the main function
if __name__ == "__main__":
    main()